In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
import pandas as pd
from Hyprid import polFeature
import numpy as np
import sys
from rdkit.Chem import Descriptors

In [3]:
def hybConn(mol):
    sp3sp3 = 0
    sp3sp2 = 0
    sp3sp = 0
    sp2sp = 0
    sp2sp2 = 0
    spsp = 0
    sp3s = 0
    sp2s = 0
    sps = 0
    other = 0
    for bond in mol.GetBonds():
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        hyb1 = str(atom1.GetHybridization())
        hyb2 = str(atom2.GetHybridization())
        if   hyb1=="SP3" and hyb2=="SP3":sp3sp3+=1
        elif hyb1=="SP3" and hyb2=="SP2":sp3sp2+=1
        elif hyb1=="SP3" and hyb2=="SP" :sp3sp+=1
        elif hyb1=="SP3" and hyb2=="S"  :sp3s+=1
        elif hyb1=="S"   and hyb2=="SP3":sp3s+=1
        elif hyb1=="SP2" and hyb2=="SP3":sp3sp2+=1
        elif hyb1=="SP2" and hyb2=="SP2":sp2sp2+=1
        elif hyb1=="SP2" and hyb2=="SP" :sp2sp+=1
        elif hyb1=="SP2" and hyb2=="S"  :sp2s+=1
        elif hyb1=="S"   and hyb2=="SP2":sp2s+=1
        elif hyb1=="SP"  and hyb2=="SP3":sp3sp+=1
        elif hyb1=="SP"  and hyb2=="SP2":sp2sp+=1
        elif hyb1=="SP"  and hyb2=="SP" :spsp+=1
        elif hyb1=="SP"  and hyb2=="S"  :sps+=1
        elif hyb1=="S"   and hyb2=="SP" :sps+=1
        else:other+=1
    return [sp3sp3,sp3sp2,sp3sp,sp3s,sp2sp2,sp2sp,sp2s,spsp,sps,other]

In [4]:
def getMolDescriptors(mol):
    res = []
    for nm,fn in Descriptors._descList:
        if nm != 'Ipc':
            try:
                val = fn(mol)
                res.append(val)
            except:
                res.append(None)
                print(nm)
    return res

In [ ]:
# Load and preprocess data
filename = "..\\Raw Files\\G12D.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename, sep='\t')
df.dropna(inplace=True)
hybd_names = ['FC','N','H','Cl','Br',
                 'I','P','F','Se','Si',
                 'S_sp3','S_sp2','S_sp',
                 'C_sp3','C_sp2','C_sp',
                 'N_sp3','N_sp2','N_sp',
                 'O_sp3','O_sp2','O_sp',
                 'I1','I2','I3']
bondhybd_names = ['sp3sp3','sp3sp2','sp3sp','sp3s',
                  'sp2sp2','sp2sp','sp2s','spsp','sps','other']
count = 0
dcr_names = [nm for nm,fn in Descriptors._descList]
dcr_names.remove('Ipc')
Cnames = ["ChEMBL ID","Smiles"]+hybd_names+bondhybd_names+dcr_names
print("Cnames", len(Cnames))
DF = pd.DataFrame(columns=Cnames)

In [5]:
filename = "C:\\Users\Gebruiker\Desktop\pycharm\G12D.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename, sep='\t')
df.dropna(subset=['IC50 (nM)'], inplace=True)
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (SwissProt) Alternative ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Alternative ID(s) of Target Chain
50,1340697,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C35H33F2N9O3/c1-2-22-24(36)6-4-18-12-...,NGNTUXWMRHHCLF-CEIJQWCGSA-N,629352,"US20230339981, Example 36",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
51,1340710,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C37H35F2N9O3/c1-2-24-26(38)6-4-20-14-...,ROZKCBLAIXNBCY-SPOXBSNYSA-N,629365,"US20230339981, Example 49",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
52,1340721,Oc1cc(-c2nc3CC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C40H35F11N6O3/c1-2-23-25(41)6-4-19-14...,NBNHXIWRJPLEJS-CKUATEJPSA-N,629376,"US20230339981, Example 63",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
53,1340735,C[Si]1(C)CCCN(CC2(COc3nc4N5C[C@H]6CC[C@H](N6)[...,InChI=1S/C39H44F2N6O2Si/c1-4-26-28(40)8-6-23-1...,TZTCEJNQRANBLJ-QGXKGNFASA-N,629390,"US20230339981, Example 78",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
54,1340738,C[C@H]1C[C@@H]2[C@@H]3CC[C@H](CN2c2nc(OC[C@@]4...,InChI=1S/C34H34F4N6O2/c1-16-9-24-23-6-4-19(39-...,VHETXHVYHQEKSW-CNTYQDABSA-N,629393,"US20230339981, Example 81",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829,1209825,Oc1cc(-c2ncc3c(nc(OC[C@@]45CCCN4C[C@H](F)C5)nc...,InChI=1S/C31H31ClF2N6O2/c32-24-4-1-3-17-9-21(4...,MREIKMRVSAOCHR-RMIANRRMSA-N,50579600,"CHEMBL4857438::US11453683, Example 251",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
3830,1340667,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C34H31F3N6O3/c1-2-21-23(36)6-4-17-10-...,MPCCWLKZACKKGV-OWBPVYLJSA-N,629322,"US20230339981, Example 2",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
3831,1340691,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C34H30F4N6O3/c1-2-20-25-16(9-22(36)27...,CDCQFERPNJQECA-KGEHGKAVSA-N,629346,"US20230339981, Example 30",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN
3832,1340693,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,InChI=1S/C36H35F3N6O3/c1-2-23-25(38)5-3-19-13-...,GSGFINJEYJSJDM-COFKYPDJSA-N,629348,"US20230339981, Example 32",GTPase KRas [G12D],NaN,NaN,0.200,...,GTPase KRas,RASK_HUMAN,P01116,A8K8Z5 B0LPF9 P01118 Q96D10,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Iterate through ChEMBL IDs and SMILES
count = 0
for i, (chembl_id, smiles) in enumerate(zip(df["BindingDB Reactant_set_id"], df["Ligand SMILES"])):
    try:
        count += 1
        if count == 100:
            count = 0
            print("Processing molecule number", i)

        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"Skipping {chembl_id}: Invalid SMILES '{smiles}'")
            continue

        mol = Chem.AddHs(mol)  # Add hydrogens
        status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())

        if status == 0:
            t = getMolDescriptors(mol)
            hyp = polFeature(mol)
            bhyp = hybConn(mol)
            I = [Descriptors3D.PMI1(mol), Descriptors3D.PMI2(mol), Descriptors3D.PMI3(mol)]

            DF.loc[i] = [chembl_id, smiles] + hyp + bhyp + I + t
            print(f"Processed {chembl_id} successfully")
        else:
            print(f"Skipping {chembl_id}: 3D conformer generation failed")

    except Exception as e:
        print(f"Error processing {chembl_id} ({smiles}): {repr(e)}")

# Save to CSV
output_filename = filename.split(".")[0] + "_Hyb_Features.csv"
DF.to_csv(output_filename, index=False)
print(f"Saved output to {output_filename}")

In [ ]:
import pandas as pd

# Load the files
df_ic50 = pd.read_csv("C:\\Users\Gebruiker\Desktop\pycharm\G12D.tsv", sep="\t")  # Read TSV file
df_features = pd.read_csv("C:\\Users\Gebruiker\Desktop\pycharm\G12D_Hyb_Features.csv")

# Merge on the respective ID columns
merged_df = df_features.merge(df_ic50[['BindingDB Reactant_set_id', 'IC50 (nM)']],
                              left_on="ChEMBL ID",
                              right_on="BindingDB Reactant_set_id",
                              how="left")

# Drop the duplicate ID column if needed
merged_df.drop(columns=["BindingDB Reactant_set_id"], inplace=True)

# Save the merged file
merged_df.to_csv("new_merged_features_IC50_g12d.csv", index=False)

print("Merge completed. File saved as merged_features_IC50_g12d.csv")
